In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import numpy as np
import datetime
import os

In [3]:
ads_data=pd.read_csv('olx_data/ads_data/ads_data.csv')
user_data=pd.read_csv('olx_data/user_data/user_data.csv')
user_msg=pd.read_csv('olx_data/user_messages.csv')
test=pd.read_csv('olx_data/user_messages_test.csv')

In [4]:
len(pd.unique(ads_data['category_id']))

10

In [5]:
test.shape
user_msg.shape
ads_data.shape
user_data.shape


(10507, 2)

(10348, 3)

(645168, 11)

(2143889, 12)

In [6]:
#no common users in test and user_msg.csv
lst1=set(test['user_id'])
len(lst1)
lst2=set(user_msg['user_id'])
len(lst2)

len(lst1.intersection(lst2))

7162

7161

0

In [7]:
test.head(2)
user_msg.head(2)
ads_data.head(2)
user_data.head(2)

,user_id,category_id
0,2,800
1,6,815


,user_id,category_id,ads
0,1,859,[1806476]
1,3,800,"[2131700, 2734107, 2877209, 2877209]"


,ad_id,category_id,seller_id,creation_time,title,description,price,lat,long,source,enabled
0,1344,800,1157,2015-11-29 13:05:26,Blackberry curve,"Funcionando, es para personal el único problem...",300.0,NaN,NaN,android,0
1,3916,815,59,2015-12-16 09:09:25,Remera marca premium alemana,Casi nuevo y sin uso. Lo compre online y me qu...,150.0,NaN,NaN,android,1


,event_time,user_id,event,channel,user_lat,user_long,origin,ad_id,images_count,ad_impressions,ad_views,ad_messages
0,2017-05-19 10:47:22,541,view,ios,-34.600439,-58.514031,browse_search,2426321,6.0,0.0,4.0,0.0
1,2017-06-10 10:18:12,501,view,android,-34.818047,-58.356583,browse_search,2746814,5.0,152.0,85.0,1.0


In [8]:
# create ads and category dataframe with ads_id as index
ind=np.array(ads_data['ad_id'])
cat=np.array(ads_data['category_id'])
ads_cat=pd.DataFrame(cat,index=ind,columns=['category_id'])

price=np.array(ads_data['price'])
ads_price=pd.DataFrame(price,index=ind,columns=['price'])

In [9]:
pd.unique(user_data['origin'])

array(['browse_search', 'browse', 'search', 'home', 'notification_center',
       'push', nan, 'drawer', 'deeplink'], dtype=object)

In [10]:
cols=['channel','user_lat','user_long']
user_data.drop(cols,axis=1,inplace=True)

mapping = {'view':0 , 'first_message':1}
user_data = user_data.replace({'event':mapping})

mapping = {'browse_search':0,'browse':0 ,'search':0,'notification_center':1, 'home':2, 'push':3, 'drawer':4, 'deeplink':4 }
user_data = user_data.replace({'origin':mapping})

user_data.dtypes

event_time         object
user_id             int64
event               int64
origin            float64
ad_id               int64
images_count      float64
ad_impressions    float64
ad_views          float64
ad_messages       float64
dtype: object

In [11]:
s1=set(test['user_id'])
s2=set(user_data['user_id'])
len(s1)
len(s1.intersection(s2))
7162-6003

7162

6003

1159

In [12]:
s1=set(user_msg.category_id)
s1
s2=set(test.category_id)
s2

{362, 800, 806, 811, 815, 853, 859, 881, 887, 888}

{362, 800, 806, 811, 815, 853, 859, 881, 887, 888}

In [13]:
user_data.dtypes

event_time         object
user_id             int64
event               int64
origin            float64
ad_id               int64
images_count      float64
ad_impressions    float64
ad_views          float64
ad_messages       float64
dtype: object

In [14]:
norm_cols=['images_count','ad_impressions','ad_views','ad_messages']#,'origin']
for c in norm_cols:
    user_data[c] = user_data[c].fillna(user_data[c].mode()[0])

In [15]:
#normaize the user_data
norm_cols=['images_count','ad_impressions','ad_views','ad_messages']#,'origin']
df=user_data[norm_cols]
df=(df-df.min())/(df.max()-df.min())
df.head()

,images_count,ad_impressions,ad_views,ad_messages
0,0.400000,0.000000,0.000518,0.000000
1,0.333333,0.000219,0.011003,0.005291
2,0.266667,0.001295,0.002848,0.005291
3,0.066667,0.000000,0.002718,0.010582
4,0.200000,0.002203,0.017087,0.015873


In [16]:
sm = df['images_count']+df['ad_impressions']+df['ad_views']+df['ad_messages']
sm = np.round(sm,2)
user_data['sum'] = sm

In [17]:
user_data = user_data.sort_values(by=['origin','sum','ad_messages','ad_views','ad_impressions'],ascending=[False,True,False,False,False])

In [18]:
ans={}
cat=pd.unique(test['category_id'])
for cid in cat:
    df1=user_data
    catdf = ads_cat.loc[ df1['ad_id'] ]
    df1 = df1[ np.array(catdf==cid) ]  #selecting those adds that have the same category as cid i.e. test instance
    pricedf = ads_price.loc[ df1['ad_id'] ]
#     df1.shape
#     pricedf.shape
    df1['price'] = np.array(pricedf)
#     df1=df1.sort_values(by=['ad_messages','ad_views','ad_impressions'],ascending=[False,False,False])

    lst=[]
    #to get unique elements in lst
    ids=list(df1['ad_id'])
    for i in np.arange(len(ids)):
        if len(lst)==10:
            break
        if not ids[i] in lst:
            lst.append(ids[i])

    ans[cid]=lst
#     print df1.columns

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [19]:
j=0
result=test[['user_id','category_id']]
res_ads=[]
for i in test.index:
    if j%100==0:
        print j
    j = j+1
    row=test.loc[i]
    uid=row['user_id']
    cid=row['category_id']
    print 'new',uid,cid
#     user_cat_ads = user_data.loc[ np.logical_and( user_data['user_id']=uid , user_data['']  )    ]
    df1 = user_data.loc[ user_data['user_id']==uid ]
    
    if df1.shape[0]==0:
        res_ads.append(str(ans[cid]))
        continue
        
    catdf = ads_cat.loc[ df1['ad_id'] ]

    df1 = df1[ np.array(catdf==cid) ]  #selecting those adds that have the same category as cid i.e. test instance
    pricedf = ads_price.loc[ df1['ad_id'] ]
    
    if df1.shape[0]>0:
        df1['price'] = np.array(pricedf)
    
#     df1=df1.sort_values(by=['ad_messages','ad_views','ad_impressions'],ascending=[False,False,False])
    
    lst=[]
    lst=list(df1['ad_id'][0:10])
    
    if len(lst)<10:
        rem=10-len(lst)
#         rem
#         ans[cid]
        lst.extend(ans[cid][0:rem])
        
    res_ads.append(str(lst))
    
#     print 'harshkara'
#     break
    

0
new 2 800
new 6 815
new 14 811
new 14 800
new 17 815
new 20 811
new 20 806
new 20 800
new 24 800
new 24 888
new 28 815
new 28 853
new 35 806
new 35 888
new 48 888
new 48 881
new 48 853
new 48 815
new 48 800
new 49 887
new 49 811
new 49 800
new 51 815
new 61 806
new 67 800
new 67 888
new 72 888
new 72 806
new 76 815
new 76 853
new 79 815
new 81 888
new 81 800
new 90 887
new 90 815
new 92 800
new 93 888
new 94 815
new 100 800
new 100 887
new 100 815
new 103 800
new 111 800
new 113 815
new 115 800
new 115 815
new 115 853
new 115 888
new 118 806
new 121 815
new 123 806
new 131 800
new 131 887
new 142 806
new 144 806
new 144 887
new 146 815
new 148 881
new 149 815
new 149 888
new 156 800
new 161 853
new 161 811
new 162 800
new 162 853
new 165 881
new 165 806
new 165 815
new 168 815
new 169 806
new 173 806
new 174 800
new 183 800
new 184 815
new 186 800
new 186 853
new 186 815
new 188 815
new 192 887
new 192 888
new 195 806
new 203 800
new 205 362
new 205 800
new 207 815
new 210 811
new 21

new 1675 806
new 1678 815
new 1678 853
new 1682 815
new 1685 806
new 1688 853
new 1688 815
new 1693 881
new 1693 853
new 1694 888
new 1701 815
new 1701 853
new 1701 887
new 1702 800
new 1705 815
new 1706 362
new 1707 800
new 1709 800
new 1710 800
new 1712 800
new 1713 888
new 1713 853
new 1713 806
new 1714 362
new 1718 853
new 1718 815
new 1724 859
new 1724 800
new 1729 800
new 1730 815
new 1730 881
new 1730 888
new 1733 815
new 1734 815
new 1736 853
new 1736 806
new 1736 815
new 1741 859
new 1749 859
new 1749 887
700
new 1749 853
new 1755 859
new 1761 806
new 1763 800
new 1764 888
new 1769 800
new 1769 806
new 1772 800
new 1772 888
new 1780 888
new 1781 888
new 1781 800
new 1781 815
new 1785 853
new 1785 806
new 1785 800
new 1792 853
new 1792 806
new 1792 888
new 1797 881
new 1797 815
new 1797 800
new 1797 806
new 1802 815
new 1803 806
new 1805 815
new 1807 815
new 1810 887
new 1812 888
new 1814 887
new 1816 859
new 1817 815
new 1824 800
new 1830 853
new 1830 800
new 1830 887
new 1830

new 3273 800
new 3277 815
new 3283 881
new 3283 853
new 3283 815
new 3283 806
new 3286 815
new 3287 800
new 3289 815
1300
new 3289 853
new 3290 800
new 3291 806
new 3299 806
new 3299 853
new 3300 888
new 3303 806
new 3303 853
new 3304 800
new 3304 806
new 3304 815
new 3304 853
new 3304 859
new 3304 888
new 3308 811
new 3320 806
new 3321 815
new 3327 800
new 3330 362
new 3332 806
new 3332 362
new 3337 815
new 3337 887
new 3345 853
new 3354 815
new 3356 888
new 3358 859
new 3359 888
new 3364 815
new 3367 800
new 3367 806
new 3374 815
new 3377 811
new 3377 815
new 3377 888
new 3377 806
new 3380 800
new 3390 800
new 3391 800
new 3391 811
new 3404 815
new 3405 800
new 3408 806
new 3408 815
new 3409 888
new 3409 800
new 3412 800
new 3413 815
new 3415 806
new 3415 815
new 3416 888
new 3416 800
new 3416 811
new 3422 815
new 3427 806
new 3427 815
new 3434 800
new 3434 815
new 3437 815
new 3438 811
new 3454 888
new 3460 881
new 3460 853
new 3460 815
new 3461 888
new 3463 853
new 3465 888
new 346

new 4906 859
new 4915 815
new 4916 800
new 4916 888
new 4916 815
new 4922 800
new 4934 887
new 4934 888
new 4935 800
new 4935 362
new 4936 815
new 4936 888
new 4937 853
new 4937 815
new 4937 800
new 4941 362
new 4941 800
new 4941 806
new 4942 806
new 4949 800
new 4951 800
new 4954 815
new 4957 853
new 4957 806
new 4959 881
new 4959 806
new 4959 887
new 4964 800
new 4967 887
new 4972 806
new 4975 887
new 4975 800
new 4980 800
new 4980 815
new 4989 800
new 4989 815
new 4993 806
new 5004 888
new 5004 806
new 5005 362
new 5009 800
new 5024 888
new 5029 806
new 5034 888
new 5034 800
new 5041 362
new 5044 887
new 5044 881
new 5047 815
new 5054 815
new 5054 800
new 5062 800
new 5067 859
new 5067 800
new 5081 806
new 5086 881
new 5089 881
new 5089 362
new 5090 806
new 5090 815
new 5094 800
new 5101 859
new 5101 800
new 5108 806
new 5114 888
new 5117 362
new 5126 815
new 5128 800
new 5128 815
new 5128 853
new 5128 888
new 5135 815
new 5138 806
new 5138 362
new 5138 881
new 5146 815
new 5146 887

new 6713 881
new 6716 806
new 6717 888
new 6718 815
new 6724 853
new 6726 888
new 6732 815
new 6738 800
new 6743 815
new 6751 853
new 6754 806
new 6759 859
new 6760 888
new 6763 800
new 6766 800
new 6767 806
new 6772 800
new 6783 800
new 6787 811
new 6791 800
new 6791 806
new 6791 815
new 6792 815
new 6792 853
new 6800 815
new 6806 888
new 6809 806
new 6810 888
new 6811 881
new 6812 800
new 6813 815
new 6821 888
new 6841 806
new 6841 815
new 6841 887
new 6849 888
new 6850 815
new 6859 815
new 6872 815
new 6873 800
new 6875 806
new 6889 811
new 6889 800
new 6896 800
new 6898 881
new 6898 800
new 6899 815
new 6899 888
new 6903 853
new 6907 800
new 6908 815
2600
new 6908 800
new 6909 888
new 6912 888
new 6920 806
new 6929 815
new 6931 800
new 6933 806
new 6935 800
new 6946 853
new 6955 815
new 6955 800
new 6956 806
new 6961 815
new 6966 853
new 6976 362
new 6977 811
new 6991 881
new 6997 815
new 7011 806
new 7015 888
new 7017 815
new 7021 815
new 7022 881
new 7027 815
new 7028 806
new 703

...

new 5632 881
new 5632 853
new 5632 800
7500
new 5635 815
new 5637 806
new 5637 853
new 5643 811
new 5644 800
new 5645 806
new 5648 806
new 5649 800
new 5655 888
new 5656 806
new 5660 853
new 5662 853
new 5670 815
new 5672 888
new 5673 806
new 5673 800
new 5673 888
new 5673 859
new 5673 853
new 5673 815
new 5674 362
new 5674 887
new 5676 362
new 5676 811
new 5676 815
new 5676 881
new 5676 888
new 5686 815
new 5689 888
new 5691 806
new 5693 800
new 5693 806
new 5695 800
new 5695 806
new 5697 362
new 5698 811
new 5698 362
new 5700 362
new 5700 800
new 5701 806
new 5701 800
new 5701 853
new 5705 815
new 5705 888
new 5708 800
new 5711 888
new 5711 806
new 5715 800
new 5722 815
new 5722 806
new 5725 888
new 5725 806
new 5732 806
new 5732 815
new 5732 881
new 5740 806
new 5742 815
new 5744 815
new 5750 800
new 5751 888
new 5761 811
new 5763 800
new 5771 800
new 5771 362
new 5779 815
new 5783 815
new 5785 806
new 5786 800
new 5788 853
new 5788 888
new 5791 853
new 5792 806
new 5792 853
new 579

new 7581 815
new 7583 815
new 7605 800
new 7606 800
new 7606 806
new 7606 881
new 7609 806
new 7609 815
new 7623 815
new 7624 815
new 7624 800
new 7627 800
new 7631 800
new 7639 887
new 7639 888
new 7643 800
new 7647 362
new 7648 806
new 7649 815
new 7654 815
new 7658 800
new 7659 800
new 7660 853
new 7660 887
new 7660 881
new 7665 881
new 7665 853
new 7665 815
new 7668 800
new 7668 811
new 7670 887
new 7672 800
new 7678 362
new 7685 800
new 7685 887
new 7690 815
new 7691 362
new 7694 881
new 7694 806
new 7697 806
new 7702 800
new 7705 806
new 7708 362
new 7708 800
new 7713 362
new 7717 800
new 7717 815
new 7718 815
new 7725 815
new 7742 806
new 7743 887
new 7743 806
new 7743 800
new 7747 800
new 7750 362
new 7752 806
new 7756 815
new 7762 815
new 7772 800
new 7774 815
new 7776 806
new 7781 815
new 7786 888
new 7788 815
new 7790 800
new 7800 806
new 7805 362
new 7806 362
new 7810 888
new 7812 806
new 7824 806
new 7825 815
new 7827 815
new 7840 859
8200
new 7840 815
new 7846 815
new 784

new 9470 853
new 9470 815
new 9479 815
new 9482 800
new 9486 362
new 9486 800
new 9494 806
new 9494 800
new 9498 800
new 9498 362
new 9501 806
new 9501 362
new 9509 800
new 9514 815
new 9514 887
new 9514 806
new 9518 362
new 9518 806
new 9519 888
new 9521 887
new 9522 806
new 9536 815
new 9539 887
new 9540 888
new 9542 806
new 9542 815
new 9548 815
new 9548 888
new 9548 853
new 9555 853
new 9558 806
new 9558 888
new 9560 815
new 9570 815
new 9575 815
new 9576 800
new 9578 800
new 9581 887
new 9581 815
new 9581 362
new 9582 800
new 9584 806
new 9588 800
new 9592 881
new 9595 806
new 9598 362
8800
new 9598 888
new 9606 800
new 9608 800
new 9611 800
new 9611 362
new 9611 806
new 9614 888
new 9614 815
new 9615 800
new 9616 806
new 9617 815
new 9621 806
new 9621 815
new 9621 881
new 9630 800
new 9637 881
new 9637 800
new 9641 806
new 9652 806
new 9654 881
new 9664 362
new 9664 800
new 9666 800
new 9667 800
new 9680 815
new 9680 362
new 9680 888
new 9680 853
new 9680 800
new 9681 800
new 968

new 11006 853
new 11009 800
new 11012 815
new 11021 888
new 11027 815
new 11028 888
new 11028 800
new 11031 881
new 11038 800
new 11046 806
new 11049 362
new 11049 815
new 11055 815
new 11055 853
new 11055 806
new 11055 800
new 11055 888
new 11061 806
new 11064 800
new 11070 806
new 11070 815
new 11073 888
new 11073 815
new 11078 362
new 11081 815
new 11084 888
new 11084 887
new 11084 800
new 11084 806
new 11084 815
new 11088 800
new 11091 853
new 11099 362
new 11100 806
new 11100 853
new 11107 815
new 11107 853
new 11107 806
new 11108 888
new 11109 800
new 11113 800
new 11113 881
new 11113 888
new 11115 888
new 11116 853
new 11123 888
9400
new 11125 815
new 11125 853
new 11129 362
new 11132 806
new 11132 800
new 11132 853
new 11135 815
new 11138 815
new 11138 853
new 11139 806
new 11140 806
new 11140 815
new 11140 800
new 11143 815
new 11143 853
new 11145 815
new 11147 815
new 11157 815
new 11164 887
new 11164 815
new 11169 800
new 11169 806
new 11170 815
new 11170 859
new 11172 806
n

new 12906 800
new 12908 806
new 12911 815
new 12915 888
new 12919 888
new 12922 362
new 12931 888
new 12932 853
new 12934 806
new 12940 888
new 12941 362
new 12944 806
new 12949 815
new 12950 815
new 12957 811
new 12966 806
new 12972 815
new 12975 815
new 12976 800
new 12977 815
new 12980 800
new 12984 811
new 12984 806
new 12988 362
new 12993 806
new 13001 815
new 13005 859
new 13006 811
new 13017 806
new 13019 806
new 13025 806
new 13027 806
new 13030 888
new 13032 806
new 13044 815
new 13047 815
new 13053 806
new 13054 853
new 13057 806
new 13066 815
new 13076 887
new 13086 811
new 13088 800
new 13088 881
new 13090 811
new 13095 881
new 13096 881
new 13099 815
new 13099 881
new 13102 859
new 13108 800
new 13108 887
new 13108 806
new 13111 800
new 13112 853
new 13112 887
new 13113 806
new 13117 811
new 13117 853
new 13120 806
10000
new 13124 800
new 13128 859
new 13143 888
new 13145 362
new 13146 815
new 13147 800
new 13148 800
new 13165 800
new 13166 815
new 13167 815
new 13170 815


In [20]:
result['ads']=res_ads

In [21]:
result.to_csv("res7.csv", index=False)
os.system("spd-say 'task complete'")

0

In [22]:
#start time 5:50

In [23]:
# lst=[1,2,3,4,5]
# lst.extend([7,8,9])
# lst

In [24]:
# list(df1['ad_id'][0:10])

In [25]:
# df['e']=df['e']+np.ones(df.shape[0],dtype=int)
# df

In [26]:
# a=np.array([[1,2,3,4,5],[6,7,8,9,10],[1,4,2,6,7],[3,1,6,9,2],[4,5,6,7,8]])
# df=pd.DataFrame(a,columns=['a','b','c','d','e'])
# df[0:100]
# b=[1,2,3]
# # ind=[0,2,4]
# df=df.loc[df['a']==1]
# df
# df['xyz']=np.array([99,100])
# df

In [27]:
# tdf=pd.DataFrame(np.array(b),index=ind,columns=['xyz'])
# tdf
# tdf==3
# oop=[7,45]
# tdf.loc[oop]

In [28]:
# df
# tdf
# tdf.loc[tdf['xyz']==3]


In [29]:
# df['xyz']=np.array(tdf)
# df

In [30]:
# df.loc[np.logical_and(df['a']==1, np.mod(df['d'],2)==1)]

In [31]:
# #ONLY FOR TESTING PURPOSE
# # uid=row['user_id']
# uid=18
# cid=800
# # cid=row['category_id']
# print uid
# print cid
# #     user_cat_ads = user_data.loc[ np.logical_and( user_data['user_id']=uid , user_data['']  )    ]
# df1 = user_data.loc[ user_data['user_id']==uid ]
# df1
# if df1.shape[0]==0:
#     df1=user_data

# catdf = ads_cat.loc[ df1['ad_id'] ]
# print '*******************************'
# catdf
# df1 = df1[ np.array(catdf==cid) ]  #selecting those adds that have the same category as cid i.e. test instance
# pricedf = ads_price.loc[ df1['ad_id'] ]

# df1['price'] = np.array(pricedf)
# df1